# Step 1: Data Loading and Initial Exploration

In [2]:
import pandas as pd

# Load the dataset
file_path = '/Users/Audrey/Desktop/megastore_dataset.csv'
df = pd.read_csv(file_path)

# Check the first few rows of the dataset
print(df.head())

# Check for missing values and data types
print(df.info())

   OrderID  CustomerID                         ProductName  Quantity  \
0   536370     12583.0         INFLATABLE POLITICAL GLOBE         48   
1   536370     12583.0      SET2 RED RETROSPOT TEA TOWELS         18   
2   536370     12583.0     PANDA AND BUNNIES STICKER SHEET        12   
3   536370     12583.0       RED TOADSTOOL LED NIGHT LIGHT        24   
4   536370     12583.0  VINTAGE HEADS AND TAILS CARD GAME         24   

      InvoiceDate  UnitPrice  TotalCost         Country Discount_Applied  \
0  12/1/2010 8:45      $0.85     $40.80   United States              Yes   
1  12/1/2010 8:45      $2.95     $53.10   United States              Yes   
2  12/1/2010 8:45      $0.85     $10.20   United States              Yes   
3  12/1/2010 8:45      $1.65     $39.60   United States              Yes   
4  12/1/2010 8:45      $1.25     $30.00   United States              Yes   

  Order Priority     Region    Segment ExpeditedShipping PaymentMethod  \
0           High  Northeast  Corpora

# Step 2: Data Wrangling (Selecting and Encoding Categorical Variables)

- Ordinal variables (Order Priority, CustomerRewardsMember)
- Nominal variables (PaymentMethod, Country)

In [3]:
from sklearn.preprocessing import LabelEncoder

# Ordinal encoding for 'Order Priority'
order_priority_mapping = {'Low': 1, 'Medium': 2, 'High': 3, 'Critical': 4}
df['Order Priority'] = df['Order Priority'].map(order_priority_mapping)

# Label encoding for 'Customer Rewards Member'
label_encoder = LabelEncoder()
df['CustomerRewardsMember'] = label_encoder.fit_transform(df['CustomerRewardsMember'])

# Print the encoded variables
print(df[['Order Priority', 'CustomerRewardsMember']].head())

   Order Priority  CustomerRewardsMember
0               3                      1
1               3                      1
2               3                      1
3               3                      1
4               3                      1


## Encoding Nominal Variables

In [4]:
# One-hot encoding for 'Payment Method' and 'Country'
df_encoded = pd.get_dummies(df, columns=['PaymentMethod', 'Country'], drop_first=True)

# Print the first few rows after encoding
print(df_encoded.head())

   OrderID  CustomerID                         ProductName  Quantity  \
0   536370     12583.0         INFLATABLE POLITICAL GLOBE         48   
1   536370     12583.0      SET2 RED RETROSPOT TEA TOWELS         18   
2   536370     12583.0     PANDA AND BUNNIES STICKER SHEET        12   
3   536370     12583.0       RED TOADSTOOL LED NIGHT LIGHT        24   
4   536370     12583.0  VINTAGE HEADS AND TAILS CARD GAME         24   

      InvoiceDate  UnitPrice  TotalCost  Discount_Applied  Order Priority  \
0  12/1/2010 8:45      $0.85     $40.80               Yes               3   
1  12/1/2010 8:45      $2.95     $53.10               Yes               3   
2  12/1/2010 8:45      $0.85     $10.20               Yes               3   
3  12/1/2010 8:45      $1.65     $39.60               Yes               3   
4  12/1/2010 8:45      $1.25     $30.00               Yes               3   

      Region    Segment ExpeditedShipping  CustomerRewardsMember  \
0  Northeast  Corporate             

# Step 3: Transactionalizing Data for Market Basket Analysis

In [6]:
# Group data by 'OrderID' and 'ProductName'
df_grouped = df.groupby(['OrderID', 'ProductName'])['Quantity'].sum().unstack().reset_index().fillna(0)

# Convert quantities to binary (0 or 1) using 'map' as per the warning
df_grouped.set_index('OrderID', inplace=True)
df_grouped = df_grouped.map(lambda x: 1 if x > 0 else 0)

# Print the first few rows of transactional data
print(df_grouped.head())

ProductName   50S CHRISTMAS GIFT BAG LARGE   DOLLY GIRL BEAKER  \
OrderID                                                          
536370                                   0                   0   
536852                                   0                   0   
536974                                   0                   0   
537065                                   0                   0   
537463                                   0                   0   

ProductName   I LOVE LONDON MINI BACKPACK   NINE DRAWER OFFICE TIDY  \
OrderID                                                               
536370                                  0                         0   
536852                                  0                         0   
536974                                  0                         0   
537065                                  0                         0   
537463                                  0                         0   

ProductName   SET 2 TEA TOWELS I LOVE L

# Step 4: Running the Apriori Algorithm

In [10]:
!pip install mlxtend

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 1.7 MB/s eta 0:00:0000:0100:010m


In [12]:
from mlxtend.frequent_patterns import apriori, association_rules

# Convert DataFrame values to boolean for compatibility
df_grouped = df_grouped.astype(bool)

# Set minimum support threshold for the Apriori algorithm
min_support = 0.05  # Example: 5% minimum support threshold

# Apply the Apriori algorithm to generate frequent itemsets
frequent_itemsets = apriori(df_grouped, min_support=min_support, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Show the generated frequent itemsets
print(frequent_itemsets)

# Show the generated association rules
print(rules)

     support                                           itemsets
0   0.063492                      (4 TRADITIONAL SPINNING TOPS)
1   0.088435                       (ALARM CLOCK BAKELIKE GREEN)
2   0.090703                        (ALARM CLOCK BAKELIKE PINK)
3   0.083900                        (ALARM CLOCK BAKELIKE RED )
4   0.061224                       (ASSORTED COLOUR MINI CASES)
..       ...                                                ...
79  0.090703  (SET6 RED SPOTTY PAPER PLATES, SET20 RED RETRO...
80  0.108844  (SET6 RED SPOTTY PAPER PLATES, SET6 RED SPOTTY...
81  0.056689  (ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI...
82  0.061224  (PLASTERS IN TIN WOODLAND ANIMALS, PLASTERS IN...
83  0.088435  (SET6 RED SPOTTY PAPER PLATES, SET6 RED SPOTTY...

[84 rows x 2 columns]
                                          antecedents  \
0                         (ALARM CLOCK BAKELIKE PINK)   
1                        (ALARM CLOCK BAKELIKE GREEN)   
2                         (ALARM CLOCK

# Step 5: Extract Support, Lift, and Confidence for Top 3 Rules

In [13]:
# Sort rules by confidence and extract the top 3 rules
top_3_rules = rules.sort_values('confidence', ascending=False).head(3)

# Print the top 3 rules with support, confidence, and lift
print(top_3_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

# You can also save the top 3 rules to a CSV if needed
top_3_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].to_csv('top_3_rules.csv', index=False)

                                          antecedents  \
49  (SET6 RED SPOTTY PAPER PLATES, SET20 RED RETRO...   
50  (SET6 RED SPOTTY PAPER CUPS, SET20 RED RETROSP...   
34                     (SET6 RED SPOTTY PAPER PLATES)   

                       consequents   support  confidence    lift  
49    (SET6 RED SPOTTY PAPER CUPS)  0.088435       0.975  7.9625  
50  (SET6 RED SPOTTY PAPER PLATES)  0.088435       0.975  8.5995  
34    (SET6 RED SPOTTY PAPER CUPS)  0.108844       0.960  7.8400  
